<a href="https://colab.research.google.com/github/MehdiKhiatiDS/DS-Unit-1-Sprint-2-Statistics/blob/master/DS-Unit-1-Sprint-2-Statistics/MK_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [45]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data






--2019-11-04 21:15:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.4’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2019-11-04 21:15:01 (127 KB/s) - ‘house-votes-84.data.4’ saved [18171/18171]



In [33]:
df = pd.read_csv('house-votes-84.data.1', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
                        

In [0]:
df =  pd.read_csv('house-votes-84.data.1', names = column_headers, na_values="?") 

In [48]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [49]:
df = df.replace({'y': 1, 'n': 0})
df.head()

df =df.replace({'y':1, 'n':0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [50]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [51]:
rep = df[df['party']=='republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [61]:
dem = df[df['party']=='democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


Null Hypothesis: There is no difference between average voting rates 
(levels of support) for the immigration bill between democrats and republicans in the house of representatives. (support is equal)
𝑥¯1==𝑥¯2 
Where  𝑥¯1  is the mean of republican votes and  𝑥¯2  is the mean of democrat votes.
Alternative Hypothesis:
𝑥¯1≠𝑥¯2 
Levels of support between the two parties will differ.95% Confidence Level



 Null Hypothesis:  𝑥¯1  (average dem support for immig bill) == .5
This says that 50% of democrats support this bill. The party is split.
Alternative Hypothesis: Support is not equal to .5 or 50%
𝑥¯1  (average dem support for immig)  ≠  .5
This says nothing about if support is greater than or less than 50%, it's just saying that it's not 50% - it's different, it's something other than 50%.
Confidence Level: 95%




In [71]:
col = rep['immigration']
np.isnan(col)
rep_immigration_no_nans = col[~np.isnan(col)]
rep_immigration_no_nans

0      1.0
1      0.0
7      0.0
8      0.0
10     0.0
      ... 
420    1.0
427    1.0
430    1.0
432    0.0
434    1.0
Name: immigration, Length: 165, dtype: float64

In [84]:
col = dem['immigration']
np.isnan(col)
dem_immigration_no_nans = col[~np.isnan(col)]
dem_immigration_no_nans

2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
      ... 
425    1.0
426    1.0
428    1.0
429    1.0
431    1.0
Name: immigration, Length: 263, dtype: float64

In [85]:
rep['immigration'].sum()/len(rep)

0.5476190476190477

In [82]:
from scipy.stats import ttest_ind
rep['immigration'].mean()


0.5575757575757576

In [90]:
rep_immigration_no_nans.mean()

0.5575757575757576

In [88]:
dem['immigration'].mean()

0.4714828897338403

In [89]:
dem_immigration_no_nans.mean()

0.4714828897338403

In [92]:
ttest_ind(rep_immigration_no_nans, dem_immigration_no_nans, nan_policy='omit')

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425282)

In [74]:
print(len(rep_immigration_no_nans))
print(len(dem_immigration_no_nans))

165
263


In [75]:
ttest_ind(rep['education'], dem['education'], nan_policy='omit')

Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

In [76]:
col = dem['education']
np.isnan(col)
dem_education_no_nans = col[~np.isnan(col)]
rep_education_no_nans = col[~np.isnan(col)]
print(len(rep_education_no_nans))
print(len(dem_education_no_nans))

249
249


In [79]:
from scipy.stats import ttest_1samp
ttest_1samp(dem['education'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=-15.915784824279635, pvalue=8.900198902856228e-40)

In [101]:
ttest_1samp(rep['education'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=13.7323961384641, pvalue=1.5754742985301713e-28)

In [98]:
ttest_1samp(dem['education'], .08, nan_policy='omit')

Ttest_1sampResult(statistic=2.891817174852164, pvalue=0.004170186036273956)

In [99]:
ttest_1samp(rep['education'], .08, nan_policy='omit')

Ttest_1sampResult(statistic=29.279856810012156, pvalue=8.0510588819391e-65)

In [80]:
dem['education'].mean()

0.14457831325301204

In [100]:
rep['education'].mean()

0.8709677419354839

In [105]:
print(ttest_1samp(dem['religious-groups'], .05, nan_policy='omit'))
ttest_1samp(rep['religious-groups'], .05, nan_policy='omit')


Ttest_1sampResult(statistic=13.697283373564195, pvalue=1.9589246266911468e-32)


Ttest_1sampResult(statistic=35.91025380673703, pvalue=6.851017575922718e-80)

In [106]:
print(ttest_1samp(dem['el-salvador-aid'], .05, nan_policy='omit'))
ttest_1samp(rep['el-salvador-aid'], .05, nan_policy='omit')

Ttest_1sampResult(statistic=6.420180364218043, pvalue=6.642559176923518e-10)


Ttest_1sampResult(statistic=53.75069433709433, pvalue=4.717313242472772e-106)

In [108]:
print(ttest_1samp(dem['right-to-sue'], .9, nan_policy='omit'))
ttest_1samp(rep['right-to-sue'], .9, nan_policy='omit')

Ttest_1sampResult(statistic=-21.315977377518696, pvalue=3.0121911713214858e-58)


Ttest_1sampResult(statistic=-1.4202366677822988, pvalue=0.1575215763850957)